In [20]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import skimage.io
import scipy as sp
import tifffile
from cellpose import plot
import pandas as pd
import glob

from urllib.parse import urlparse
from cellpose import models
import skimage.measure as measure
import skimage.transform as transform
import plotly.graph_objs as go

import cv2
import napari
import scipy.ndimage as ndimage
from roifile import ImagejRoi
from cellpose import models
import glob
import plotly.express as px
from scipy import stats
import skimage as ski

In [21]:
viewer = napari.Viewer()

In [22]:
wwidth=65
wheight=130


In [23]:
def merge_rDNA_labels(rDNA_labels, chromosome_labels):
    rtn_labels = np.zeros(rDNA_labels.shape)
    for a in range(1,np.max(rDNA_labels)+1):
        sub = chromosome_labels[rDNA_labels==a]
        candidate = stats.mode(sub[sub>0])[0]
        if np.isnan(candidate):
            candidate = 0
        rtn_labels[rDNA_labels==a] = candidate
    return rtn_labels.astype(int)

# Analysis (no rois)

We're going to have a go at just using the marker intensity, this is complicated by the fact that Chromosome 2 actually has 4 pieces instead of just 2.

In [24]:
df = pd.read_csv('Results.csv')
df = df.fillna(0)

In [25]:
df = df.merge(df.groupby(['File']).agg({'Areas':len}).reset_index().rename(columns={'Areas':'ChromCount'}), on='File')
df = df[df['ChromCount']>=18]

In [26]:
df

,Unnamed: 0,File,IntDens,rDNA_Areas,Areas,Mean,Marker_Mean,UBF_Mean,UBF_IntDens,labels,Chromosome,ChromCount
0,14,./acrocentric_inferred\B.Orang_chr13_UBF_1001-...,0.074889,129,441.0,589.550388,3117.4377,2648432.0,0.223231,15.0,chr13,18
1,6,./acrocentric_inferred\B.Orang_chr13_UBF_1001-...,0.110767,150,475.0,749.913333,3101.4336,3426991.0,0.288854,7.0,chr13,18
2,8,./acrocentric_inferred\B.Orang_chr13_UBF_1001-...,0.014362,93,460.0,156.827957,2178.2173,-7312.0,-0.000616,9.0,chr13,18
3,11,./acrocentric_inferred\B.Orang_chr13_UBF_1001-...,0.000000,0,333.0,0.000000,2159.1230,0.0,0.000000,12.0,chr13,18
4,17,./acrocentric_inferred\B.Orang_chr13_UBF_1001-...,0.024165,68,509.0,360.882353,2143.4520,183059.0,0.015430,18.0,chr13,18
...,...,...,...,...,...,...,...,...,...,...,...,...
3025,10,./acrocentric_inferred\B.Orang_chr9_UBF_1035-1...,0.080987,102,410.0,692.833333,2486.5293,173986.0,0.063036,11.0,chr9,18
3026,7,./acrocentric_inferred\B.Orang_chr9_UBF_1035-1...,0.254754,189,467.0,1176.169312,2476.8093,-2622.0,-0.000950,8.0,chr9,18
3027,3,./acrocentric_inferred\B.Orang_chr9_UBF_1035-1...,0.024907,68,349.0,319.617647,2446.9856,138608.0,0.050218,4.0,chr9,18
3028,9,./acrocentric_inferred\B.Orang_chr9_UBF_1035-1...,0.095712,140,500.0,596.557143,2443.4780,241533.0,0.087509,10.0,chr9,18


In [27]:
df['MarkerRank'] = df.groupby(['File']).rank(ascending=False)['Marker_Mean']

In [28]:
sub_df = df[df['MarkerRank']<3]
chr2_df = df[(df['MarkerRank']>2) & (df['MarkerRank']<5) & (df['Chromosome']=='chr2')]

In [29]:
combined_df = pd.concat([sub_df, chr2_df])
combined_df

,Unnamed: 0,File,IntDens,rDNA_Areas,Areas,Mean,Marker_Mean,UBF_Mean,UBF_IntDens,labels,Chromosome,ChromCount,MarkerRank
0,14,./acrocentric_inferred\B.Orang_chr13_UBF_1001-...,0.074889,129,441.0,589.550388,3117.4377,2648432.0,0.223231,15.0,chr13,18,1.0
1,6,./acrocentric_inferred\B.Orang_chr13_UBF_1001-...,0.110767,150,475.0,749.913333,3101.4336,3426991.0,0.288854,7.0,chr13,18,2.0
18,13,./acrocentric_inferred\B.Orang_chr13_UBF_1002-...,0.105508,167,562.0,803.161677,3574.3718,839983.0,0.191019,14.0,chr13,18,1.0
19,0,./acrocentric_inferred\B.Orang_chr13_UBF_1002-...,0.074928,124,531.0,768.169355,3543.2354,1187958.0,0.270152,1.0,chr13,18,2.0
36,15,./acrocentric_inferred\B.Orang_chr13_UBF_1003-...,0.063970,84,580.0,805.750000,2975.4312,1017432.0,0.111017,16.0,chr13,18,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2505,0,./acrocentric_inferred\B.Orang_chr2_UBF_1024-1...,0.089535,69,456.0,1557.565217,3670.0986,714491.0,0.210838,1.0,chr2,18,4.0
2522,4,./acrocentric_inferred\B.Orang_chr2_UBF_1025-1...,0.016688,89,494.0,199.044944,4265.5510,-1822.0,-0.000912,5.0,chr2,18,3.0
2523,1,./acrocentric_inferred\B.Orang_chr2_UBF_1025-1...,0.046181,80,522.0,612.775000,3881.4368,-9869.0,-0.004941,2.0,chr2,18,4.0
2540,4,./acrocentric_inferred\B.Orang_chr2_UBF_1026-1...,0.040228,95,457.0,577.800000,6445.4180,2223.0,0.000655,5.0,chr2,18,3.0


In [30]:
combined_df['rDNARank'] = combined_df.groupby(['File']).rank(ascending=False)['IntDens']

In [31]:

combined_df.to_csv('CombinedResults.csv')

# Visualize

In [32]:
f = px.box(combined_df, x='Chromosome', y='IntDens', points='all', color='rDNARank', hover_data='File',
           category_orders={'rDNARank':[1.0, 2.0, 3.0, 4.0], 'Chromosome':['chr2', 'chr9', 'chr13', 'chr14', 'chr15', 'chr18', 'chr21', 'chr22']})
f.write_html('rDNA.html')
f

### Summing total intensity (copy number)

In [33]:
combined_df.groupby(['Chromosome', 'rDNARank']).agg({'IntDens': np.median}).reset_index()['IntDens'].sum()

C:\Users\smc\AppData\Local\Temp\ipykernel_22784\593718832.py:1: FutureWarning:

The provided callable <function median at 0x000002012B620700> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.



1.0229668289006797

This should be close to 1.0

In [43]:
combined_df.groupby(['Chromosome', 'rDNARank']).agg({'IntDens': np.median}).reset_index()

C:\Users\smc\AppData\Local\Temp\ipykernel_22784\615909716.py:1: FutureWarning:

The provided callable <function median at 0x000002012B620700> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.



,Chromosome,rDNARank,IntDens
0,chr13,1.0,0.104519
1,chr13,1.5,0.000000
2,chr13,2.0,0.074444
3,chr14,1.0,0.070597
4,chr14,2.0,0.052033
5,chr15,1.0,0.209813
6,chr15,2.0,0.105769
7,chr18,1.0,0.047820
8,chr18,1.5,0.000000
9,chr18,2.0,0.000000


### Interactive versions

In [34]:
def examine_file(fname, labeled_chromosomes):
    print(fname)
    img = tifffile.imread(fname)
    chromosomes = img[-1].astype(int)
    img = img[0:-1]
    rDNA = tifffile.imread(fname.replace('acrocentric_inferred', 'rDNA_inferred'))[-1].astype(int)
    rDNA_merged = merge_rDNA_labels(rDNA, chromosomes)

    marker_img = np.zeros_like(chromosomes)
    for c in labeled_chromosomes:
        marker_img[chromosomes==c] = 1
    marker_img = ski.morphology.binary_dilation(marker_img, ski.morphology.disk(3))
    marker_img = ski.feature.canny(marker_img)

    viewer.layers.clear()
    viewer.add_image(img, channel_axis=0, name='img')
    viewer.add_labels(chromosomes, name='chromosomes')
    viewer.layers[-1].contour = 2
    viewer.add_labels(rDNA, name='rDNA', visible=False)
    viewer.add_labels(rDNA_merged, name='rDNA_merged')
    viewer.add_image(marker_img, name='marker', blending='additive', colormap='gray', opacity=0.4)

In [35]:
f=go.FigureWidget(
    px.box(combined_df, x='Chromosome', y='IntDens', points='all', color='rDNARank', hover_data='File',
           category_orders={'rDNARank':[1.0, 2.0, 3.0, 4.0], 'Chromosome':['chr2', 'chr9', 'chr13', 'chr14', 'chr15', 'chr18', 'chr21', 'chr22']})
    )

def click_fn(trace, points, state):
    
    if (len(points.point_inds)>0):
        idx = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(idx)
        cdf = combined_df[combined_df['File']==idx]
        examine_file(idx, cdf['labels'].values)

for a in f.data:
    a.on_click(click_fn)

f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'customdata': array([['./acrocentric_inferred\\B.Orang_chr13_UBF_1001-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1002-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1003-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1006-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1009-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1012-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1013-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1014-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1015-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13

./acrocentric_inferred\B.Orang_chr14_UBF_1007-1.tif
./acrocentric_inferred\B.Orang_chr14_UBF_1007-1.tif
./acrocentric_inferred\B.Orang_chr14_UBF_1005-1.tif
./acrocentric_inferred\B.Orang_chr14_UBF_1005-1.tif
./acrocentric_inferred\B.Orang_chr14_UBF_1016-1.tif
./acrocentric_inferred\B.Orang_chr14_UBF_1016-1.tif


# Visualize UBF

In [36]:
combined_UBF_df = combined_df.copy()
combined_UBF_df['UBFRank'] = combined_df.groupby(['File']).rank(ascending=False)['UBF_IntDens']

In [37]:
f = px.box(combined_df, x='Chromosome', y='UBF_IntDens', points='all', color='rDNARank', hover_data='File',
           category_orders={'rDNARank':[1.0, 2.0, 3.0, 4.0], 'Chromosome':['chr2', 'chr9', 'chr13', 'chr14', 'chr15', 'chr18', 'chr21', 'chr22']})
f.write_html('UBF.html')
f

In [47]:
f = px.box(combined_UBF_df, x='Chromosome', y='UBF_IntDens', points='all', color='UBFRank', hover_data='File',
           category_orders={'UBFRank':[1.0, 2.0, 3.0, 4.0], 'Chromosome':['chr2', 'chr9', 'chr13', 'chr14', 'chr15', 'chr18', 'chr21', 'chr22']})
f.write_html('UBF_reranked.html')
f

In [48]:
f=go.FigureWidget(
    px.box(combined_UBF_df, x='Chromosome', y='UBF_IntDens', points='all', color='UBFRank', hover_data='File',
           category_orders={'UBFRank':[1.0, 2.0, 3.0, 4.0], 'Chromosome':['chr2', 'chr9', 'chr13', 'chr14', 'chr15', 'chr18', 'chr21', 'chr22']})
    )

def click_fn(trace, points, state):
    
    if (len(points.point_inds)>0):
        idx = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(idx)
        cdf = combined_df[combined_df['File']==idx]
        examine_file(idx, cdf['labels'].values)

for a in f.data:
    a.on_click(click_fn)

f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'customdata': array([['./acrocentric_inferred\\B.Orang_chr13_UBF_1001-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1002-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1003-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1006-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1009-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1012-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1013-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1014-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13_UBF_1015-1.tif'],
                                   ['./acrocentric_inferred\\B.Orang_chr13

./acrocentric_inferred\B.Orang_chr14_UBF_1005-1.tif
./acrocentric_inferred\B.Orang_chr14_UBF_1005-1.tif


In [40]:
combined_df[combined_df['File']=='./acrocentric_inferred\B.Orang_chr14_UBF_1014-1.tif']

,Unnamed: 0,File,IntDens,rDNA_Areas,Areas,Mean,Marker_Mean,UBF_Mean,UBF_IntDens,labels,Chromosome,ChromCount,MarkerRank,rDNARank
563,16,./acrocentric_inferred\B.Orang_chr14_UBF_1014-...,0.051277,92,347.0,790.804348,3688.3198,19911.0,0.002600,17.0,chr14,19,1.0,1.0
564,7,./acrocentric_inferred\B.Orang_chr14_UBF_1014-...,0.047302,80,359.0,838.912500,3369.5989,793940.0,0.103672,8.0,chr14,19,2.0,2.0


In [41]:
px.scatter(combined_df, x='IntDens', y='UBF_IntDens', animation_frame='Chromosome', range_x=[-.02,0.34], range_y=[-.02,.44], color='rDNARank')